In [10]:
X=[[0],[1],[2],[3]]
y1=[0,0,1,1]
from sklearn.neighbors import KNeighborsClassifier
neigh=KNeighborsClassifier(n_neighbors=3,metric="euclidean")
neigh.fit(X,y1)
print('Knn predefinie-classification:',neigh.predict([[2.1]]))
from sklearn.neighbors import KNeighborsRegressor
knn_regression=KNeighborsRegressor(3)
y2=[15.3,8.6,6.5,4.29]
knn_regression.fit(X,y2)
print('knn predefinie-regression:',knn_regression.predict([[2.1]]))

Knn predefinie-classification: [1]
knn predefinie-regression: [6.46333333]


In [19]:
from collections import Counter
from numpy import *
import math
def mean(labels):
    """calcul de la moyenne des k étiquettes"""
    return sum(labels)/len(labels)
def mode(labels):
    """calcul du mode des k étiquettes"""
    return Counter(labels).most_common(1)[0][0]
def euclidean_distance(point1,point2):
    """calcul de la distance euclidienne"""
    sum_squared_distance=0
    for i in range(len(point1)):
        sum_squared_distance+=math.pow(point1[i]-point2[i],2)
    return math.sqrt(sum_squared_distance)



In [43]:
from collections import Counter
from numpy import *
import math
def mean(labels):
    """calcul de la moyenne des k étiquettes"""
    return sum(labels)/len(labels)
def mode(labels):
    """calcul du mode des k étiquettes"""
    return Counter(labels).most_common(1)[0][0]
def euclidean_distance(point1,point2):
    """calcul de la distance euclidienne"""
    sum_squared_distance=0
    for i in range(len(point1)):
        sum_squared_distance+=math.pow(point1[i]-point2[i],2)
    return math.sqrt(sum_squared_distance)

def knn(data,query,k,distance_utilisee,choix_knn_regress_ou_classif):
    """Algorithme des k plus proches voisins KNN"""
    distances_et_positions=[]
    for index,example in enumerate(data):
        distance=distance_utilisee(example[:-1],query)
        distances_et_positions.append((distance,index))
    sorted_distances_et_positions=sorted(distances_et_positions)
    k_proches_distances_and_indices=sorted_distances_et_positions[:k]
    k_prohes_classes=[]
    for dist_indice in k_proches_distances_and_indices:
        num=dist_indice[1];k_prohes_classes+=[data[num][1]]
    return k_proches_distances_and_indices

        
a=array([[1,2,1],[1,0,1],[2,0,1],[1,1,1],[0,1,0],[2,6,5]])
b=knn(a,[1,1,1],3,euclidean_distance,mean)
print(b)
for i in range(len(b)):
    print(a[b[i][1]])     
        
    


In [45]:
from numpy import *
import math
a=array([[1,2,1],[1,0,1],[2,0,1],[1,1,1],[0,1,0],[2,6,5]])
b=knn(a,[1,1,1],3,euclidean_distance,mean)
print(b)
for i in range(len(b)):
    print(a[b[i][1]])

[(0.0, 3), (1.0, 0), (1.0, 1)]
[1 1 1]
[1 2 1]
[1 0 1]


In [ ]:
import numpy as np
from numpy import random
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA as pca_dec

from plot_utils import pca_scree_plot
import pickle as pk

from data import Data


class Pca:
    def init(self, data: Data, load=True, pkl_pca_path="pca_None.pkl", var_exp=None, k=60):
        self.data = data
        if load:  # load pca model
            self.pca = pk.load(open(pkl_pca_path, 'rb'))
        else:
            # Apply PCA to extract eigenvectors
            self.pca = pca_dec(var_exp).fit(self.data.flat_scans)
            # save PCA model
            pk.dump(self.pca, open(pkl_pca_path, "wb"))
        # Getting the cumulative variance
        self.var_cumu = np.cumsum(self.pca.explained_variance_ratio_) * 100

        # define number of components
        if not var_exp:
            self.k = k
        else:
            self.k = pca.cumulative_Explained_Variance(var_exp)

        # Take the first K principal components as eigenvectors
        self.eigenmesh = self.k_principal_components(self.k)

        # Generate weights as a KxN matrix where K is the number of eigenvectors and N the number of samples
        self.weights = self.eigenmesh @ (self.data.flat_scans - self.pca.mean_).T  # new data
        # mean and standard deviation
        self.mu = self.weights.mean()
        self.sigma = self.weights.std()

        # print("Weights shape: ",self.weights.shape)
        print("PCA Model loaded/cretedt successfully!")

    def cumulative_Explained_Variance(self, threshold=95, show_fig=False):
# -- creating scree plot --
        # How many PCs explain 95% of the variance?
        k = np.argmax(self.var_cumu > threshold) + 1
        print(f"Number of components explaining {threshold}% variance: " + str(k))
        # print("\n")
        pca_scree_plot(self.var_cumu, k, threshold, show_fig)
        return k

    def k_principal_components(self, k):
        # get related eigen data to k components
        eigenmesh = self.pca.components_[:k]
        # print(f'Shape of eigenmesh : {eigenmesh.shape}')
        return eigenmesh

    def random_weights(self, mu, sigma, k):
        # generate random weights with a normal distribution
        return random.normal(mu, sigma, k)

    def create_random(self, k, mu=None, sigma=None, eigenmesh=None, data_mean=None):
        # create random 3d mesh
        # weights = self.random_weights(mu, sigma, k)
        weights = np.zeros(k)
        newscan = weights @ eigenmesh + data_mean
        newscan = newscan.reshape(self.data.scan_shape)
        return newscan

In [11]:
import pandas
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
data=pandas.read_csv("images_chiffres_codees_niveau_de_gris.csv")
data=np.array(data);x=data[0:1000,1:];y=data[0:1000,0];
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
model=KNeighborsClassifier(n_neighbors=5,metric='manhattan')
model.fit(x_train,y_train);score=model.score(x_test,y_test)
print("score sur test-set par k=5 -->"+str(score*100)+'%')
distances=['euclidean','manhattan']
valeurs_de_k=np.arange(1,8)
parametres_grid={"n_neighbors":valeurs_de_k,'metric':distances}
model=KNeighborsClassifier()
grid=GridSearchCV(model,parametres_grid,cv=3)


score sur test-set par k=5 -->85.0%
